# Demo notebook showing arbok functionalities

This is a simple demo notebook containing the core functionalities of arbok

### Importing qm and arbok module

In [1]:
from qm.qua import *
from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.simulate.credentials import create_credentials
from qm import SimulationConfig

2023-09-08 15:02:20,853 - qm - INFO     - Starting session: c6614b6a-c0a6-467b-849d-e428d0121782


In [2]:
from arbok.core.sequence import Sequence
from arbok.core.program import Program
from arbok.core.sample import Sample

from arbok.samples.sunshine.readout.other_ST_read import OtherStReadout
from arbok.samples.sunshine.initialization.mixed_down_up_init import (
    MixedDownUpInit
)
from arbok.samples.sunshine.control.smart_Y import SmartY
from arbok.samples.sunshine.control.smart_swap import SmartSwap
from arbok.samples.sunshine.control.dummy_control import Square
from arbok.samples.sunshine.configs.rf2v_config import rf2v_config

### Instanciating Arbok modules

Firstly we instanciate all `Sequence` objects that should build the entire 
measurement. Since we imported all sequences from the 'arbok.samples.sunshine' 
they already have the config for that specific sample

In [3]:
smart_Y = SmartY('smartY')
smart_swap12 = SmartSwap('smartSwap12')
smart_swap23 = SmartSwap('smartSwap32')
square = Square('square')
duInit = MixedDownUpInit('duInit')
stReadout = OtherStReadout('stReadout')

Next we create an empty `Sequence` class we call 'MEAS' that will structure all 
subsequeces. This time we need to pass a `Sample` since its a unconfigured 
sequence. We create second empty sequence to summarize all control sequences 
(everything happening before readout and after initialization). This is not
mandatory but is helpfull if you want to simulate the entire qubit control part
of the experiment.

In [4]:
sunshine = Sample('sunshine', rf2v_config)

In [5]:
MEAS = Program('MEAS', sample = sunshine)

In [6]:
MEAS.connect_OPX(host_ip= '192.168.0.3')

2023-09-08 15:02:21,606 - qm - INFO     - Performing health check
2023-09-08 15:02:21,647 - qm - INFO     - Health check passed


In [7]:
#MEAS = Sequence('MEAS', sample = sunshine)
CYY = Sequence('CYY', sample = sunshine)

Thus we add all sub-sequences responsible for qubit control in 'CYY' ...

In [8]:
CYY.add_subsequence(smart_swap12)
CYY.add_subsequence(smart_Y)
CYY.add_subsequence(smart_swap23)

... and finally add initialization, control and readout to the summarizing 
`Sequence`. 

In [10]:
MEAS.add_subsequence(duInit)
MEAS.add_subsequence(CYY)
MEAS.add_subsequence(square)
MEAS.add_subsequence(stReadout)

KeyError: 'Duplicate submodule name duInit'

In [11]:
amp_range =  np.arange(0.5, 1, 0.05)
time_range =  np.arange(4, 105, 1, dtype=int)

In [12]:
MEAS.settables = [MEAS.square.amp, MEAS.square.tSquare]
MEAS.setpoints_grid = [ amp_range, time_range ]

In [13]:
print( len(amp_range), len(time_range) )

10 101


In [13]:
#MEAS.duInit.tPreControl(int(2e2/4))
#MEAS.duInit.tInitLoadMixed(int(1e2/4))
#MEAS.duInit.tPreControlRampMixed(int(1e2/4))

In [14]:
#MEAS.run_remote_simulation(duration=8000)

In [15]:
#MEAS.run_remote_simulation(duration=8000)

In [16]:
MEAS.run()

2023-07-25 09:29:17,024 - qm - INFO     - Sending program to QOP for compilation
2023-07-25 09:29:18,773 - qm - INFO     - Executing program


In [17]:
#while MEAS.result_handles.is_processing():
#    MEAS.stReadout.read_state()

### Now putting everything in quantify

In [18]:
import quantify_core.data.handling as dh
from quantify_core.measurement import MeasurementControl

In [19]:
dh.set_datadir(dh.default_datadir())

Data will be saved in:
C:\Users\labuser\quantify-data


In [20]:
meas_ctrl = MeasurementControl('meas_ctrl')

In [21]:
meas_ctrl.gettables(MEAS.stReadout.read_state)

In [22]:
meas_ctrl.settables([MEAS.square.amp, MEAS.square.tSquare])

In [23]:
meas_ctrl.setpoints_grid([amp_range, time_range])

In [24]:
arr = MEAS.stReadout.read_state.get()

In [25]:
arr

In [26]:
MEAS.sweep_len

1010

In [27]:
arr = MEAS.stReadout.read_state()

busy 
 paused;  False
 total counts: 334
 registerd; 1010
busy 
 paused;  False
 total counts: 334
 registerd; 1010
busy 
 paused;  False
 total counts: 354
 registerd; 1010
busy 
 paused;  False
 total counts: 354
 registerd; 1010
busy 
 paused;  False
 total counts: 374
 registerd; 1010
busy 
 paused;  False
 total counts: 374
 registerd; 1010
busy 
 paused;  False
 total counts: 394
 registerd; 1010
busy 
 paused;  False
 total counts: 394
 registerd; 1010
busy 
 paused;  False
 total counts: 394
 registerd; 1010
busy 
 paused;  False
 total counts: 414
 registerd; 1010
busy 
 paused;  False
 total counts: 414
 registerd; 1010
busy 
 paused;  False
 total counts: 438
 registerd; 1010
busy 
 paused;  False
 total counts: 438
 registerd; 1010
busy 
 paused;  False
 total counts: 458
 registerd; 1010
busy 
 paused;  False
 total counts: 458
 registerd; 1010
busy 
 paused;  False
 total counts: 478
 registerd; 1010
busy 
 paused;  False
 total counts: 478
 registerd; 1010
busy 
 paused;

In [28]:
MEAS.stReadout.read_state.batch_size = 1010
MEAS.square.amp.batch_size = 1010
MEAS.square.tSquare.batch_size = 1010

In [31]:
type(arr)

NoneType

In [29]:
meas_ctrl.run()

Starting batched measurement...
Iterative settable(s) [outer loop(s)]:
	 --- (None) --- 
Batched settable(s):
	 amp, tSquare 
Batch size limit: 1010

busy 
 paused;  False
 total counts: 1012
 registerd; 2020
busy 
 paused;  False
 total counts: 1036
 registerd; 2020
busy 
 paused;  False
 total counts: 1036
 registerd; 2020
busy 
 paused;  False
 total counts: 1056
 registerd; 2020
busy 
 paused;  False
 total counts: 1056
 registerd; 2020
busy 
 paused;  False
 total counts: 1076
 registerd; 2020
busy 
 paused;  False
 total counts: 1076
 registerd; 2020
busy 
 paused;  False
 total counts: 1076
 registerd; 2020
busy 
 paused;  False
 total counts: 1096
 registerd; 2020
busy 
 paused;  False
 total counts: 1096
 registerd; 2020
busy 
 paused;  False
 total counts: 1116
 registerd; 2020
busy 
 paused;  False
 total counts: 1116
 registerd; 2020
busy 
 paused;  False
 total counts: 1136
 registerd; 2020
busy 
 paused;  False
 total counts: 1136
 registerd; 2020
busy 
 paused;  False
 t

TypeError: 'NoneType' object is not iterable

In [ ]:
res.count_so_far()/MEAS.sweep_size

In [ ]:
while MEAS.qm_job.is_paused():
    print(MEAS.result_handles.read_state.count_so_far(), end = '\r')

In [ ]:
import time
last_time = time.time()
last_size = np.size(MEAS.result_handles.get('read_state').fetch_all())
time_list = []
res_list = []
MEAS.qm_job.resume()

while MEAS.result_handles.is_processing():
    if not MEAS.qm_job.is_paused():
        print('Running:',end='\r')
    if MEAS.qm_job.is_paused():
        now_time = time.time()
        diff_time = now_time-last_time
        time_list.append(diff_time)
        last_size = np.size(res.fetch_all())
        diff_res = np.size(MEAS.result_handles.get('diff_state').fetch_all())- last_size
        res_list.append(diff_res)
        shape = np.shape(MEAS.result_handles.read_state_run.fetch_all())
        if True:
            print(
                shape, last_size, 
                np.mean(res_list),
                MEAS.qm_job.is_paused(), 
                np.mean(time_list), end='\r')
        last_size = np.size(MEAS.result_handles.get('diff_state').fetch_all())
        last_time = time.time()
        
        MEAS.qm_job.resume()
    

In [ ]:
MEAS.result_handles.is_processing()

In [ ]:
MEAS.opx.close()

In [ ]:
#MEAS.result_handles.get('read_state').fetch_all()

Note that the QCoDeS `Parameter`s describing the respective sub sequenes are 
saved within their sub classes. The QUA code lives within the sub classes as well
and gets compiled recursively, once we call `get_program()` on the class itself
or on any of its parent sequences. See in the cell below how to access parameters.

In [ ]:
MEAS.CYY.smartSwap12.tSwap.get()

In [ ]:
MEAS.stReadout.vPreRead_P1.get()

In [ ]:
MEAS.CYY.submodules

In [ ]:
MEAS.CYY.smartSwap12.root_instrument

### Simulating Sequences

The entire sequence, any sub group or any sub sequence can now be simulated by
calling `run_remote_simulation` with the desired simulation duration in cycles 
(4ns/cycle). Simulating $\sqrt{SWAP}$ ...

In [ ]:
job = MEAS.CYY.smartSwap12.run_remote_simulation(duration = 1000)

... and the entire $CYY$-gate:

In [ ]:
MEAS.CYY.smartY.smart_cycles(1)
job = MEAS.CYY.run_remote_simulation(duration = 2000)

### Parameter sweeps

Next, we demonstrate parameter sweeps within the QUA script. Choose a `sequence`
you want to simulate and pass a list of QCoDeS parameters you want to sweep, as
well as a numpy array for each of those parameters. The last parameter in the
list corresponds to the innermost loop of the sweep. In the following we only
simulate $\sqrt{SWAP}$ gate while sweeping the delay and duration of the swap
pulse.

In [ ]:
MEAS.square.settables = [MEAS.square.amp, MEAS.square.tSquare]
MEAS.square.setpoints_grid = [ np.linspace(0.1, 1, 3), np.linspace(4, 50, 3) ]

In [ ]:
job = MEAS.square.run_remote_simulation(duration = 850)

We can simulate the same parameter sweep for the summarizing `Sequence` 'CYY' as
well, for example to see how pulses align for previous or following sequences.
To do that, we have to add `settables` and the `setpoints_grid` to the sequence
we want to simulate.

In [ ]:
MEAS.CYY.settables = [MEAS.CYY.smartSwap12.vControlSWAP_J1, MEAS.CYY.smartSwap12.tSwap]
MEAS.CYY.setpoints_grid = [ np.linspace(0.5, 1, 3), np.linspace(4, 100, 3) ]
MEAS.CYY.smartSwap12.vControlSWAP_J1.batched = True
MEAS.CYY.smartSwap12.tSwap.batched = True
MEAS.CYY.smartY.smart_cycles(1)

In [ ]:
job = MEAS.CYY.run_remote_simulation(duration = 12000)

### Simulating readouts

In [ ]:
MEAS.CYY.smartSwap12.tWait(int(192))
#job = MEAS.CYY.smartSwap12.run_remote_simulation(duration = 2000)
job = MEAS.stReadout.run_remote_simulation(duration = 3000)

In [ ]:
samples = job.get_simulated_samples()

In [ ]:
job.result_handles.__dict__

In [ ]:
job.result_handles.diff_state.fetch_all()['value']

### Using the `Sequence` in the quantify-core library

In [ ]:
import quantify_core.data.handling as dh
from quantify_core.measurement import MeasurementControl

In [ ]:
dh.set_datadir(dh.default_datadir())

In [ ]:
meas_ctrl = MeasurementControl('meas_ctrl')

In [ ]:
import quantify_core.visualization.pyqt_plotmon as pqm
from quantify_core.visualization.instrument_monitor import InstrumentMonitor

In [ ]:
# plotmon = pqm.PlotMonitor_pyqt("plotmon")
# insmon = InstrumentMonitor("InstrumentMonitor")
# meas_ctrl.instr_plotmon(plotmon.name)

In [ ]:
import datetime

MEAS.stReadout.add_parameter(
    name = 'dummy_read',
    unit = '',
    get_cmd = lambda: np.cos(datetime.datetime.now().microsecond*1e-4)
)

In [ ]:
meas_ctrl.settables([
    MEAS.CYY.smartSwap12.tWait, 
    MEAS.CYY.smartSwap12.vControlSWAP_J1
])
meas_ctrl.setpoints_grid([
    range(180,220), 
    range(4,2000)
])

meas_ctrl.gettables(MEAS.stReadout.dummy_read)
#dset = meas_ctrl.run(name="Frequency sweep")

Gettables need to be `ParameterWithSetpoints`. This takes an arbitrary amount of 1D setpoint arrays (sizes (n), (m), (k)) and returns (n,m,k) get array.

In [ ]:
#meas_ctrl.run()

Note that 'tInitLoadMixed' and 'tPreControl' have been reduced for the sake of visibility.

In [ ]:
arr = np.zeros((4,5))